<a href="https://colab.research.google.com/github/rodrigonevest/web-scraping/blob/main/Scraping_Cepea_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import urllib3

In [ ]:
#Site base usado no web scraping
base = 'https://www.cepea.esalq.usp.br/en/category/brazilian-agribusiness-news.aspx'

#Pegando os valores das datas primeiramente.
res = requests.get(base)
index = BeautifulSoup(res.content,'lxml')
pag = index.find('select',{'name':'data'})
pag = pag.find_all('option')
test = list()

#Salvando em uma lista dos valores extraindo, para posteriormente percorrer as paginas
for w in pag:
  test.append(w['value']) 
#Total de paginas que precisa percorrer
print(len(test))

#Criando DataFrame
df = pd.DataFrame(columns=('Date','Text_Headline','Text_Body','Commodity','Language'))

#Percorrendo a lista de paginas.
for x in test:
  #Realizando o request junto com o {x} que foi obtido através do for percorrendo as datas da paginação
  r = requests.get(f'{base}?data={x}')
  
  #Chamando o BeautifulSoup e pegando seu conteudo do request, e extraindo a data e link
  soup = BeautifulSoup(r.content,'lxml')
  noticiaslista = soup.find('div',{'class':'imagenet-conteudo'})
  data = noticiaslista.find('option',{'selected':'selected'}).get_text()
  noticiaslista = noticiaslista.find_all('a')
  
  #Percorrendo a tag <a> para extrair as noticias, machetes e commodity.
  for link in noticiaslista:     
    
    #Extraindo o link.
    aux = str(link['href'])
     
    #Extraindo a commodity .  
    cm = link.find('strong').get_text()
    #Extraindo a manchete.
    hd = link.find('p').get_text()
    #Realizando o request do link extraido.
    dados = requests.get(aux)
    soup = BeautifulSoup(dados.content,'lxml')

    #Extraindo todo o texto da noticia do link extraido.
    conteudo = soup.find('div',{'class':'imagenet-ma-b imagenet-fl imagenet-col-12 imagenet-table-responsiva'})
    if conteudo != None:
      #Realizando alguns ajuste no texto extraido.
      conteudo = [x.get_text(strip=True) for x in soup.find_all('p')]
      conteudo = str(conteudo)
      conteudo = conteudo.replace("['",'')
      conteudo = conteudo.replace("']",'')
      conteudo = conteudo.replace('["','')
      conteudo = conteudo.replace('"]','')
      conteudo = conteudo.replace("', '",'')     
    
    
    #Definindo os Parametros
    df = df.append({'Date':data,
                    'Language': 'En',
                    'Text_Headline': hd,
                    'Text_Body': conteudo,
                    'Commodity': cm}, ignore_index=True)      

  

360


In [ ]:
df.isnull().sum()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.head(-1)

,Date,Text_Headline,Text_Body,Commodity,Language
0,2021-06-01,Number of animals slaughtered in BR is the low...,"Cepea, June 1st, 2021CATTLE –Fed cattle prices...","Meat (beef, pork, and poultry)",En
1,2021-06-01,Weather issues may constrain the recovery of t...,"Cepea, June 1st, 2021 – Estimates released on ...",Citrus,En
2,2021-06-01,"Despite the beginning of the harvesting, arabi...","Cepea, June 1st, 2021 – The harvesting of the ...",Coffee,En
3,2021-06-01,"Purchasers put pressure, and cotton quotes dro...","Cepea, June 1st – Cotton prices faded in the B...",Cotton,En
4,2021-06-01,Price paid to dairy farmers in BR rises for th...,"Cepea, June 1st, 2021 – The prices paid to Bra...",Dairy,En
5,2021-06-01,Higher demand in May boosts prices in the Braz...,"Cepea, June 1st, 2021 – After having decreased...",Ethanol,En
6,2021-06-01,Dollar depreciation favors imports,"Cepea, June 1st, 2021 – The recent depreciatio...",Wheat,En
7,2021-06-01,May's average price sets a nominal record in t...,"Cepea, June 1st, 2021 – Crystal sugar prices h...",Sugar,En
8,2021-06-01,"Despite the devaluations late in the month, In...","Cepea, June 1st, 2021 – Corn prices dropped in...",Corn,En
9,2021-06-01,"Brazilian processors reduce purchases, and dom...","Cepea, June 1st, 2021 – Soybean prices faded i...",Soybean,En


In [ ]:
df.to_excel('Scraping_Cepea_20pg-17.xlsx', index=False)